In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Setting a seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

# Load data
df = pd.read_csv('Data/Annotations_final.csv')

# Preprocessing steps (if any)

# Grouping data for cross-validation
group_dict = dict()
groups = np.array([])

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])

# Set up the splitter
gkf = GroupKFold(n_splits=5)

# Extract features and labels
X = df['annotation_text']
y = df['comment_answer']

# Storage for performance metrics
roc_auc_scores = np.array([])

# SMOTE for balancing classes
smote = SMOTE(random_state=SEED)

# Hyperparameters
num_epochs = 50  # Adjust as needed
batch_size = 16  # Adjust as needed
learning_rate = 1e-4  # Adjust as needed
dropout_rate = 0.5  # Adjust as needed

# Model Training and Evaluation
for train_index, test_index in gkf.split(X, y, groups=groups):
    try:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Convert texts to embeddings
        training_embeddings = embed(X_train.to_list())
        testing_embeddings = embed(X_test.to_list())

        # Apply SMOTE
        X_sm, y_sm = smote.fit_resample(training_embeddings.numpy(), y_train)

        # Define the model
        model = Sequential([
            Dense(64, activation='relu', input_shape=(512,), kernel_regularizer=l2(0.01)),
            Dropout(dropout_rate),
            Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
            Dropout(dropout_rate),
            Dense(1, activation='sigmoid')
        ])

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Model training
        model.fit(X_sm, y_sm, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

        # Model evaluation
        predictions = model.predict(testing_embeddings)
        roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))
    except Exception as e:
        print(f"An error occurred: {e}")

# Output the results
print(f"Average ROC AUC Score: {np.round(roc_auc_scores.mean(), 3)}")
print(f"ROC AUC Score Standard Deviation: {np.round(roc_auc_scores.std(), 3)}")
print(f"Maximum ROC AUC Score: {np.round(roc_auc_scores.max(), 3)}")


Epoch 1/50



19/19 [==============================] - 1s 10ms/step - loss: 2.2394 - accuracy: 0.5189 - val_loss: 2.2122 - val_accuracy: 0.4242
Epoch 2/50
19/19 [==============================] - 0s 2ms/step - loss: 2.1794 - accuracy: 0.5498 - val_loss: 2.1515 - val_accuracy: 0.6667
Epoch 3/50
19/19 [==============================] - 0s 2ms/step - loss: 2.1251 - accuracy: 0.5911 - val_loss: 2.0940 - val_accuracy: 0.7879
Epoch 4/50
19/19 [==============================] - 0s 2ms/step - loss: 2.0724 - accuracy: 0.5979 - val_loss: 2.0392 - val_accuracy: 0.8182
Epoch 5/50
19/19 [==============================] - 0s 2ms/step - loss: 2.0191 - accuracy: 0.6392 - val_loss: 1.9889 - val_accuracy: 0.7879
Epoch 6/50
19/19 [==============================] - 0s 2ms/step - loss: 1.9688 - accuracy: 0.6770 - val_loss: 1.9405 - val_accuracy: 0.7879
Epoch 7/50
19/19 [==============================] - 0s 2ms/step - loss: 1.9181 - accuracy: 0.7182 - val_loss: 1.8903 - val_accuracy: 0.8182
Epoch 8/50
19/19 [============

2/2 [==============================] - 0s 0s/step
Average ROC AUC Score: 0.963
ROC AUC Score Standard Deviation: 0.015
Maximum ROC AUC Score: 0.982
